In [5]:
import sys
import os

# Add the root folder to Python's module search path
sys.path.append(os.path.abspath(".."))

from resnet_vqgan import VQGan
from resnet_encoder import ResNetEncoderConfig
from resnet_decoder import ResNetDecoderConfig
from quantizer import QuantizerConfig
import torch.nn.functional as F
from lpips import LPIPS
from discriminator import Discriminator, DiscriminatorConfig
import torch
import torch.nn.functional as F
disc = Discriminator (DiscriminatorConfig)
import warnings
warnings.filterwarnings('always')


disc.to('cuda')
percept_disc = LPIPS().eval()
percept_disc.to('cuda')
x = torch.randn (1, 3, 256,256).to('cuda')
gan = VQGan ()
gan.to('cuda')

master_process = True
steps_per_epoch = 500
d_loss_factor = 1.0
grad_accum_steps = 4
lr = 6e-4

c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Zen\miniconda3\envs\tr\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
e:\Research\vincentVanGogh\resnet_experiment\lpips.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURIT

In [2]:
def compute_loss (vqgan_model, discriminator, perceptual_distinguisher, x, perceptual_loss_factor, rec_loss_factor, current_step):
    print ("\n inside compute loss \n")
    reconstructed_iamges, encoding_indices, vq_loss = vqgan_model (x)
    print ("\n vqgan model forwded\n")
    # Discriminator Forward
    disc_real = discriminator (x)
    disc_generated = discriminator (reconstructed_iamges)
    print ("\nDISC forwarded\n")
    # VGG forward
    perceptual_loss = perceptual_distinguisher (x, reconstructed_iamges) # (B, 1, 1, 1)
    perceptual_loss = perceptual_loss.squeeze ( ) #(B)
    print ("\VGG FORWARDED\n")
    # formality none of the optimizers have access to vgg params but this way its explicit and efficient
    perceptual_loss = perceptual_loss.detach ( )

    reconstruction_loss = F.mse_loss (x, reconstructed_iamges, reduction='none') #(B, C, H, W)
    reconstruction_loss = reconstruction_loss.mean (dim=(1,2,3)) #(B)
    print ("\Rec loss computed\n")
    perceptual_recon_loss = rec_loss_factor * reconstruction_loss + perceptual_loss_factor * perceptual_loss
    perceptual_recon_loss = perceptual_recon_loss.mean()
    print ("\ncombined perceptual rec loss computed\n")

    # compute adversarial loss for generator if it's not zeroth epoch (VQGAN paper specs)
    is_zeroth_epoch = True if current_step < steps_per_epoch else False
    if is_zeroth_epoch:
        # don't calculate
        # Wasserstein loss, approximation of BCE (discriminator(generated), ones_like(generated))
        # TODO: Internalize throughly
        # good approximation of KLD between discriminator's predictions given generated inputs and ideal distribution of generated images being classified as real.
        # basically mean negative log likelihood is approximation of F.cross_entropy (discriminator(generated), ones_like(generated)) which is approximation of 
        # KLD between (discriminator(generated), ones_like(generated)) which is proportional to
        # BCE (discriminator(genrated), ones_like(generated))
        g_loss = 0 # replace by formula
        print ("\nlambda skipped\n")
    else:
        #calculate adversarial loss for generator
        g_loss = - torch.mean (disc_generated)
        lambda_factor = vqgan_model.compute_lambda (perceptual_recon_loss, g_loss)
        g_loss = lambda_factor * g_loss
        if master_process:
            print (f"\nGanLoss{g_loss:.6f} activated!\n")
        
    # VQ GAN LOSS
    # Net Generator Loss
    vq_gan_loss = perceptual_recon_loss + vq_loss + g_loss
    print ("VQGAN LOSS COMPUTED")
                    
    # Discriminator Loss : Hinge Loss. Push Logits for the catgeorical distribution that come out of discriminator 
    # to be more than +1 for real images
    # At the same time Push logits for categorical distribution that comes out of discriminator to be less than -1 for fake images

    # drive logits corresponding to real images from discriminator above zero
    # don't need to explicitly squeeze out the spurious depth dimension (1 channels) from discriminator output, mean will still be evaluated correctly
    d_loss_real= torch.mean(F.relu (1.0 - disc_real))
    # drive logits corresponding to fake(generate) images from discriminator below -1
    d_loss_fake = torch.mean(F.relu (1.0 + disc_generated))

    d_loss = d_loss_factor * 0.5 * (d_loss_real + d_loss_fake)
    return vq_gan_loss, d_loss

<>:12: DeprecationWarning: invalid escape sequence \V
<>:18: DeprecationWarning: invalid escape sequence \R
<>:12: DeprecationWarning: invalid escape sequence \V
<>:18: DeprecationWarning: invalid escape sequence \R
C:\Users\Zen\AppData\Local\Temp\ipykernel_29580\1499343910.py:12: DeprecationWarning: invalid escape sequence \V
  print ("\VGG FORWARDED\n")
C:\Users\Zen\AppData\Local\Temp\ipykernel_29580\1499343910.py:18: DeprecationWarning: invalid escape sequence \R
  print ("\Rec loss computed\n")


In [3]:
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os

ddp = int (os.environ.get ('RANK', -1)) != -1 # is this a ddp run or not?
if ddp:
    assert torch.cuda.is_available(), f"CUDA is required for DDP"
    init_process_group (backend='nccl')
    ddp_rank = int (os.environ['RANK'])
    ddp_local_rank = int (os.environ['LOCAL_RANK'])
    ddp_world_size = int (os.environ['WORLD_SIZE'])
    device = f"cuda:{ddp_local_rank}"
    torch.cuda.set_device (device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc
else:
    # non ddp vanilla run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    elif hasattr (torch.backends, 'mps') and torch.backends.mps.is_available ():
        device = 'mps'
    print (f"using device: {device}")

ddp = int (os.environ.get ('RANK', -1)) != -1 # is this a ddp run or not?
if ddp:
    assert torch.cuda.is_available(), f"CUDA is required for DDP"
    init_process_group (backend='nccl')
    ddp_rank = int (os.environ['RANK'])
    ddp_local_rank = int (os.environ['LOCAL_RANK'])
    ddp_world_size = int (os.environ['WORLD_SIZE'])
    device = f"cuda:{ddp_local_rank}"
    torch.cuda.set_device (device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc
else:
    # non ddp vanilla run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    elif hasattr (torch.backends, 'mps') and torch.backends.mps.is_available ():
        device = 'mps'
    print (f"using device: {device}")

using device: cuda
using device: cuda


In [4]:
if ddp:
    # forward is unchanged, backward is mostly unchanged except there is overlap between computation and communication of gradients
    # while the backward pass is still going on, to average the gradients from all processes
    # we're tacking on this average as we will see in a bit
    gan = DDP (gan, device_ids=[ddp_local_rank])
    disc = DDP (disc, device_ids=[ddp_local_rank])
raw_gan = gan.module if ddp else gan
raw_discriminator = disc.module if ddp else disc

In [5]:
gan_opt = raw_gan.configure_optimizers (0.1, 6e-4, 'cuda')
gan.train()
gan_opt.zero_grad()
vqgan_loss_accum = 0.0

optimizer_configuration for VQGAN model:
 num decayed parameter tensors:50 with 104367168 parameters
num non-decayed parameter tensors:105 with 54467 parameters
using fused AdamW:True


In [6]:
for micro_step in range (grad_accum_steps):
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        vqgan_loss, dloss = compute_loss (gan, disc, percept_disc, x, 1.0, 1.0, 500)
    vqgan_loss = vqgan_loss / grad_accum_steps
    vqgan_loss_accum += vqgan_loss.detach()

    is_last_micro_step = (micro_step == (grad_accum_steps - 1))
    if ddp:
        # very last backward will have the grad_sync flag as True
        # for now this works, but not a good practice if pytorch takes the flag away
        # averages gradients
        gan.require_backward_grad_sync = is_last_micro_step
        disc.require_backward_grad_sync = is_last_micro_step
    
    vqgan_loss.backward (retain_graph=True)
    dloss.backward()

if ddp:
        # calculates average of loss_accum on all the ranks, and it deposits that average on all the ranks
        # all the ranks will contain loss_accum averaged up
        dist.all_reduce (vqgan_loss_accum, op= dist.ReduceOp.AVG)
vqgan_norm = torch.nn.utils.clip_grad_norm_ (gan.parameters(), 1.0)


 inside compute loss 


	ze computed


	applying quantizer 

on
torch.Size([1, 512, 16, 16])

	 rearranged ze for computing L2 with codebook


	 calculating distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	 calculated distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	quantizer applied


 vqgan model forwded


DISC forwarded

\VGG FORWARDED

\Rec loss computed


combined perceptual rec loss computed


GanLoss-1.209179 activated!

VQGAN LOSS COMPUTED

 inside compute loss 


	ze computed


	applying quantizer 

on
torch.Size([1, 512, 16, 16])

	 rearranged ze for computing L2 with codebook


	 calculating distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	 calculated distances for each of torch.Size([256, 1, 512]) with codebook vectors torch.Size([1024, 512])


	quantizer applied


 vqgan model forwded


DISC forwarded

\VGG FORWARDED

\Rec loss co

In [7]:
for param_group in gan_opt.param_groups:
        param_group['lr'] = lr

In [8]:
gan_opt.step()